# 爬虫实验

## 1.1-1.7 学习

In [ ]:
import urllib.request

response = urllib.request.urlopen('https://www.baidu.com')
print(response.read().decode('utf-8'))

In [ ]:
import urllib.request
import urllib.parse

data = bytes(urllib.parse.urlencode({'Chritch': '3200104207'}), encoding='utf8')
response = urllib.request.urlopen('https://httpbin.org/post', data=data)
print(response.read().decode('utf-8'))

In [ ]:
from lxml import etree

text = '''
<html><body><div>
<ul>
<li class="0"> first  </li>
<li class="1"> second </li>
<li class="1"> third  </li>
<li class="0"> fourth </li>
</ul>
</div></body></html>
'''

html = etree.HTML(text)
result = html.xpath('//li')
print(result)

In [ ]:
from lxml import etree

text = '''
<html><body><div>
<ul>
<li class="0"> first  </li>
<li class="1"> second </li>
<li class="1"> third  </li>
<li class="0"> fourth </li>
</ul>
</div></body></html>
'''

html = etree.HTML(text)
result = html.xpath('//li[@class="0"]')
print(result)


In [ ]:
from lxml import etree 

text = '''
<html><body><div>
<ul>
<li class="0"> first  </li>
<li class="1"> second </li>
<li class="1"> third  </li>
<li class="0"> fourth </li>
</ul>
</div></body></html>
'''

html = etree.HTML(text)
result = html.xpath('//li[@class="0"]/text()')
print(result)

## 1.8 爬取猫眼电影 top100

### 1.8.1 爬取 html
因为猫眼电影近些年的反爬机制变严格了，故采取了一些比较特殊的手段

- 根据文章指示，在谷歌浏览器调试模式下，copy as cURL(bash)，然后使用 curl-converter 转换成 python 代码
- 注意：如果爬取的内容为输入验证信息，则需要现在浏览器中验证后，再重复上述步骤（保证没有被猫眼ban掉）
- references
    - https://blog.csdn.net/weixin_40340586/article/details/120134102
    - https://curlconverter.com/


In [ ]:
import requests
import time


cookies = {
    '__mta': '217962025.1672026973241.1672052779855.1672052791315.27',
    'uuid_n_v': 'v1',
    'uuid': '3D291FD084D111EDBB2321DFA394E4C280D7CD55FEE54EFD943630063FB059E1',
    '_csrf': '65bfbba146b96ab86c672c8aa5062a9350ff0a51eedbb3ce81a2af58b2c3ad96',
    '_lxsdk_cuid': '181aa0bc24713-09a91303c0c072-26021a51-240000-181aa0bc248c8',
    '_lxsdk': '3D291FD084D111EDBB2321DFA394E4C280D7CD55FEE54EFD943630063FB059E1',
    'Hm_lvt_703e94591e87be68cc8da0da7cbd0be2': '1672026972',
    '__mta': '142375437.1672026977996.1672026977996.1672026977996.1',
    'Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2': '1672052791',
    '_lxsdk_s': '1854e1b83af-6b6-637-298%7C%7C11',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'zh,zh-TW;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': '__mta=217962025.1672026973241.1672052779855.1672052791315.27; uuid_n_v=v1; uuid=3D291FD084D111EDBB2321DFA394E4C280D7CD55FEE54EFD943630063FB059E1; _csrf=65bfbba146b96ab86c672c8aa5062a9350ff0a51eedbb3ce81a2af58b2c3ad96; _lxsdk_cuid=181aa0bc24713-09a91303c0c072-26021a51-240000-181aa0bc248c8; _lxsdk=3D291FD084D111EDBB2321DFA394E4C280D7CD55FEE54EFD943630063FB059E1; Hm_lvt_703e94591e87be68cc8da0da7cbd0be2=1672026972; __mta=142375437.1672026977996.1672026977996.1672026977996.1; Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2=1672052791; _lxsdk_s=1854e1b83af-6b6-637-298%7C%7C11',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}


pages = []
for i in range(10):
    # 使用 cookies 以绕过猫眼的验证，offset为当前页的第一部电影排名
    response = requests.get('https://www.maoyan.com/board/4?offset={}'.format(i*10), cookies=cookies, headers=headers)
    pages.append(response.text)
    print("已获取第{}页".format(i+1))
    # 间隔 3 秒时间，防止过于频繁
    time.sleep(3)

### 1.8.2 提取 top100 电影信息

In [ ]:
from lxml import etree

top_movies = [] 

def extract_element(text):
    print(text)
    html = etree.HTML(text)
    # 提取各种元素
    numbers = html.xpath('//div[@class="content"]/div/div/dl/dd/i/text()')
    movies = html.xpath('//div[@class="content"]/div/div/dl/dd/div/div/div/p/a/text()')
    stars = html.xpath('//div[@class="content"]/div/div/dl/dd/div/div/div/p[@class="star"]/text()')
    releasetimes = html.xpath('//div[@class="content"]/div/div/dl/dd/div/div/div/p[@class="releasetime"]/text()')
    scores_interger = html.xpath('//div[@class="content"]/div/div/dl/dd/div/div/div/p[@class="score"]/i[@class="integer"]/text()')
    scores_fraction = html.xpath('//div[@class="content"]/div/div/dl/dd/div/div/div/p[@class="score"]/i[@class="fraction"]/text()')
    for i in range(10):
        movie = []
        movie.append(numbers[i])
        movie.append(movies[i])
        movie.append(stars[i].replace('\n', '').strip())
        movie.append(releasetimes[i])
        movie.append(scores_interger[i] + scores_fraction[i])
        # 加入top movies列表中
        top_movies.append(movie)

# 解析1-10页的内容
for i in range(10):
    extract_element(pages[i])

### 1.8.3 输出到文件

In [ ]:
open('top_100.txt', 'w').close()
for i in range(100):
    with open('top_100.txt', 'a') as f:
        print(top_movies[i], file=f)

## 2 OCR 识别

安装 tesserocr 的库时，可能需要一些库支持
```bash
$ sudo apt install libleptonica-dev libtesseract-dev tesseract-ocr{,-eng,-osd}
```

In [ ]:
import tesserocr
from PIL import Image
from matplotlib import pyplot as plt

image = Image.open('checkcode.png')

# 处理图像
image = image.convert('L')
threshold = 150 # 设置滤波上限
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)
image = image.point(table, '1')

# plt.imshow(image)
result = tesserocr.image_to_text(image)
print(result)

## 3 爬取起点中文网免费小说

In [21]:
import lxml.html
from lxml import etree
import requests
import urllib
import random
import time
import random
def get_ua():
    # https://blog.csdn.net/SL_World/article/details/99647119
    first_num = random.randint(55, 76)
    third_num = random.randint(0, 3800)
    fourth_num = random.randint(0, 140)
    os_type = [
        '(Windows NT 6.1; WOW64)', '(Windows NT 10.0; WOW64)', '(X11; Linux x86_64)',
        '(Machintosh; Inter Mac OS X 10_14_5)'
    ]
    chrome_version = 'Chrome/{}.0.{}.{}'.format(first_num, third_num, fourth_num)
    ua = ' '.join(['Mozilla/5.0', random.choice(os_type), 'AppleWebKit/537.36',
    '(KHTML, like Gecko)', chrome_version, 'Safari/537.36'])
    return ua

def get_html(url):

    headers = {
        'Referer': 'https://book.qidian.com/',
        'User-Agent': get_ua(),
    }

    # response = requests.get('https://book.qidian.com/info/1034919102/', cookies=cookies, headers=headers)

    try: 
        res = requests.get(url,headers=headers, timeout=(3,4))
        try:
            res.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(f'Error: {err}')
        else: 
            print('Success!')
        return res.text
    except:
        print(res.text)
        print("爬取超时")
# 输入小说的第一章
url = 'https://read.qidian.com/chapter/0mNMNoFQxR3bhZU9AFSCzA2/RqJBDW3NgsP4p8iEw--PPw2/'

# pages = []
for i in range(30):
    print("爬取第{}章".format(i+1))
    res = get_html(url)
    # print(res)
    html = etree.HTML(res)
    pages.append(html)
    url = "https:{}".format(html.xpath('//*[@id="j_chapterNext"]/@href')[0])
    print("获取链接：{}".format(url))
    time.sleep(random.random()*3)
    # break
# catalog_page = lxml.html.document_fromstring(get_html(url))
# print(catalog_page.text)


爬取第1章
Success!
    <script>"" id="j-readPage">
    var name = "seo-jump-referrer";
    try {
        var reg = new RegExp("(^| )" + name + "=([^;]*)(;|$)");
        var seoReferer = document.cookie.match(reg);
        if(seoReferer && seoReferer[2]){
            Object.defineProperty(document, "referrer", {
                value: decodeURIComponent(seoReferer[2])
            });
            document.cookie = name+'=;domain=.qidian.com;path=/;expires=Thu, 01-Jan-1970 00:00:01 GMT';
        }
    } catch (error) {
        console.log(error);
    }
    if (window.outerWidth <= 1024 && navigator.userAgent.match(/(Android|iPhone|SymbianOS|Windows Phone|iPad|iPod|HarmonyOS)/i) && !navigator.userAgent.match(/spider/i)) {
        var url = getMUrl();
        if (url) {
            if (location.search) {
                if (/source=m_jump/.test(location.search)) {
                    url += location.search.replace("source=m_jump", "source=pc_jump")
                } else if (/source=pc_jump/.te

In [24]:
from lxml import etree
# print(catalog_page)
open('novel.txt', 'w').close()
for html in pages:
    text = html.xpath('//div[@class="read-content j_readContent"]/p/text()')
    for line in text:
        with open('novel.txt', 'a') as f:
            print(line, file=f)
    # print(text)

遇到的问题：
- 动态加载的主页 -> 采用访问第一页，然后顺序向后索引的方式
- 获取的html和原有的html内容不一致 -> 参考获取的 html 页内容

In [ ]:
import  requests
from fake_useragent import UserAgent
from lxml import etree
class photo_spider(object):
    def __init__(self):
        self.url = 'https://book.qidian.com/info/1020580616#Catalog'
        ua = UserAgent(verify_ssl=False)
        #随机产生user-agent
        for i in range(1, 100):
            self.headers = {
                'User-Agent': ua.random
            }
    def mian(self):
     pass
def get_html(self,url):
    response=requests.get(url,headers=self.headers)
    html=response.content.decode('utf-8')
    return html
class qidian(object):
    def __init__(self):
        self.url = 'https://book.qidian.com/info/1020580616#Catalog'
        ua = UserAgent(verify_ssl=False)
        for i in range(1, 100):
            self.headers = {
                'User-Agent': ua.random
            }
    def get_html(self,url):
        response=requests.get(url,headers=self.headers)
        html=response.content.decode('utf-8')
        return html
    def parse_html(self,html):
        target=etree.HTML(html)
        links=target.xpath('//ul[@class="cf"]/li/a/@href')#获取链接
        names=target.xpath('//ul[@class="cf"]/li/a/text()')#获取每一章的名字
        for link,name in zip(links,names):
            print(name+'\t'+'https:'+link)
    def main(self):
        url=self.url
        html=self.get_html(url)
        self.parse_html(html)
        print(html)
spider=qidian()
spider.main()